In [7]:
import os,sys,datetime
import matplotlib.pyplot as plt
sys.path.append(os.getcwd())

#Module imports
import geeViz.getImagesLib as getImagesLib
import geeViz.taskManagerLib as taskManagerLib
import geeViz.changeDetectionLib as changeDetectionLib
ee = getImagesLib.ee
Map = getImagesLib.Map
Map.clearMap()
##################################################

mtbs = ee.ImageCollection("USFS/GTAC/MTBS/annual_burn_severity_mosaics/v1")
##################################################

tcc = ee.ImageCollection('projects/nlcd-tcc/assets/CONUS-TCC/Final-products/tcc_rawAnalytic')
tcc = tcc.filter(ee.Filter.stringContains('system:index','_mean'))
tcc_years = tcc.aggregate_histogram('Year').keys().map(lambda n:ee.Number.parse(n).int16()).getInfo()
tcc = ee.ImageCollection([tcc.filter(ee.Filter.calendarRange(yr,yr,'year')).mosaic().set('system:time_start',ee.Date.fromYMD(yr,6,1).millis()) for yr in tcc_years])
##################################################
# Min number of years from all available TCC years to start and stop with the MTBS collection
pre_min = 2
post_min = 5

mtbs_years = tcc_years[pre_min:-post_min+1]
print('All TCC Years:',tcc_years)
print('Included MTBS Years:',mtbs_years)

# Iterate through each mtbs year and then tcc year
# Mask the tcc year by the mtbs burn mask for the given mtbs year and set relevant properties
zCollection = []
for mtbs_year in mtbs_years:
    mtbs_yr = mtbs.filter(ee.Filter.calendarRange(mtbs_year,mtbs_year,'year')).mosaic()
    for tcc_year in tcc_years:
        zYr = tcc_year-mtbs_year
        tcc_yr = tcc.filter(ee.Filter.calendarRange(tcc_year,tcc_year,'year')).mosaic().updateMask(mtbs_yr)
        tcc_yr = tcc_yr.set({'tcc_year':tcc_year,'mtbs_year':mtbs_year,'zYr':zYr})
        zCollection.append(ee.Image(tcc_yr))
        print(mtbs_year,tcc_year,zYr)
zCollection = ee.ImageCollection(zCollection)

# Find the z years available and mosaic based on those years
zYrs = zCollection.aggregate_histogram('zYr').keys().map(lambda n:ee.Number.parse(n).int16()).sort()
def getZYr(zYr):
	zYr = ee.Number(zYr)
	zCYr = zCollection.filter(ee.Filter.eq('zYr',zYr)).mosaic()
	d = ee.Date.fromYMD(zYr.add(1900),6,1).millis()
	return zCYr.set({'zYr':zYr,'system:time_start':d})
zCollection = ee.ImageCollection(zYrs.map(getZYr))

Map.addTimeLapse(zCollection,{'min':0,'max':80,'palette':'EEE,080'},'TCC Z MTBS')

summaryArea = ee.Geometry.Polygon(
          [[[-116.35310363752222, 44.63158113370513],
          [-116.35310363752222, 43.351201397586905],
          [-113.62849426252222, 43.351201397586905],
          [-113.62849426252222, 44.63158113370513]]], None, False)
		
zBns = zYrs.map(lambda bn:ee.Number(bn).format())
print(zBns.getInfo())
tccStack = zCollection.toBands().rename(zBns)
print('The stack band names are: {}'.format(tccStack.bandNames().getInfo()))

# Get zonal mean
tccMeans = tccStack.reduceRegion(ee.Reducer.mean(), summaryArea, 30, 'EPSG:5070', None, True, 1e13, 4).getInfo() #gee reduceRegion applies a reducer to all pixels
tccStdDev = tccStack.reduceRegion(ee.Reducer.stdDev(), summaryArea, 30, 'EPSG:5070', None, True, 1e13, 4).getInfo() #gee reduceRegion applies a reducer to all pixels
tccCount = tccStack.reduceRegion(ee.Reducer.count(), summaryArea, 30, 'EPSG:5070', None, True, 1e13, 4).getInfo() #gee reduceRegion applies a reducer to all pixels
print(tccCount)
print(tccMeans.keys())
# plt.plot([str(i) for i in zYrs.getInfo()],tccMeans.values())

Map.turnOnInspector()
# Map.view()

All TCC Years: [2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]
Included MTBS Years: [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]
2010 2008 -2
2010 2009 -1
2010 2010 0
2010 2011 1
2010 2012 2
2010 2013 3
2010 2014 4
2010 2015 5
2010 2016 6
2010 2017 7
2010 2018 8
2010 2019 9
2010 2020 10
2010 2021 11
2011 2008 -3
2011 2009 -2
2011 2010 -1
2011 2011 0
2011 2012 1
2011 2013 2
2011 2014 3
2011 2015 4
2011 2016 5
2011 2017 6
2011 2018 7
2011 2019 8
2011 2020 9
2011 2021 10
2012 2008 -4
2012 2009 -3
2012 2010 -2
2012 2011 -1
2012 2012 0
2012 2013 1
2012 2014 2
2012 2015 3
2012 2016 4
2012 2017 5
2012 2018 6
2012 2019 7
2012 2020 8
2012 2021 9
2013 2008 -5
2013 2009 -4
2013 2010 -3
2013 2011 -2
2013 2012 -1
2013 2013 0
2013 2014 1
2013 2015 2
2013 2016 3
2013 2017 4
2013 2018 5
2013 2019 6
2013 2020 7
2013 2021 8
2014 2008 -6
2014 2009 -5
2014 2010 -4
2014 2011 -3
2014 2012 -2
2014 2013 -1
2014 2014 0
2014 2015 1
2014 2016 2
2014 2017 3
2014 2018 4
201